In [3]:
from src.helpers import *
from sklearn.pipeline import Pipeline
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import time
from umap import UMAP

2024-01-04 20:31:28.618446: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 20:31:28.646652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 20:31:28.646674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 20:31:28.647348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 20:31:28.651842: I tensorflow/core/platform/cpu_feature_guar

In [4]:
# Reload the saved scaled data
X_train = pd.read_csv('data/train/scaled_X_train.csv', parse_dates=True, index_col='unix')
y_train = pd.read_csv('data/train/y_train.csv', parse_dates=True, index_col='unix')
X_test = pd.read_csv('data/test/scaled_X_test.csv', parse_dates=True, index_col='unix')
y_test = pd.read_csv('data/test/y_test.csv', parse_dates=True, index_col='unix')
X_val = pd.read_csv('data/val/scaled_X_val.csv', parse_dates=True, index_col='unix')
y_val = pd.read_csv('data/val/y_val.csv', parse_dates=True, index_col='unix')

In [5]:
X_train.shape

(28500, 324)

In [6]:
start = time.time()
for i in range(1, 1000):
    continue
end = time.time()
duration = '{0:.5f}'.format(end - start)
print(f'{duration} seconds')

0.00006 seconds


In [7]:
# Calculate the weights for the imbalanced classes
y = pd.concat([y_train, y_val, y_test])
weights = cwts(y.values.flatten())


In [8]:
rf = RandomForestClassifier(n_jobs=-1, class_weight=weights)

In [9]:
# Define the various feature selection methods
vif = VIFTransform(dropna=True)
boruta = BorutaPy(rf, n_estimators='auto', verbose=2)

In [10]:
# Define data pipelines
pipelines = [
    'none',
    Pipeline([('vif', vif)])
]

In [11]:
umap = UMAP(n_neighbors=5)
pipe_1 = Pipeline([('umap', umap)])

In [12]:
filepath = './tensorboard/feature_selection/run'
time = datetime.now().strftime('%m-%d-%Y-%H:%M:%S')
for key in pipe_1.named_steps.keys():
    filepath += f'_{key}_'
filepath += time
filepath

'./tensorboard/feature_selection/run_umap_01-04-2024-20:31:34'

In [13]:
X_train = pipe_1.fit_transform(X_train, y_train.values.ravel())

/home/boshlike/.local/share/virtualenvs/final_assignment_code-eTELY0YF/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [14]:
X_train

array([[-0.13059509, -0.25891238],
       [ 9.091423  , -1.1986651 ],
       [ 9.092318  , -1.1952734 ],
       ...,
       [-7.7361903 ,  9.329575  ],
       [-7.739735  ,  9.327072  ],
       [-7.7416525 ,  9.327802  ]], dtype=float32)